# LU разложение, блочные и ленточные матрицы

Существует множество типов матриц, удовлетворяющих некоторым дополнительным условиям, для которых многие матричные операции могут быть вычислены быстрее или точнее, чем для матриц произвольного вида. 
В данной лабораторной мы начнем писать библиотеку на Python, которая будет содержать классы, реализующие базовые алгоритмы для работы с основными типами матриц.
Далее приводится исходный код класса `Matrix`, являющегося общим предком для всех матриц, и реализующего логику работы с матрицами общего вида.
Нижеследующий класс `FullMatrix` реализует хранилище для заполненных матриц. 
Изучите эти реализации и выполните следующие задания:

1. Напишите метод `lu` для класса `Matrix`, выполняющий LU разложение. 
2. Реализуйте метод `det`, вычисляющий определитель матрицы, опираясь на LU разложение.
3. Реализация `FullMatrix` может содержать своими элементами другие матрицы, т.е. описывать блочную матрицу. Убедитесь, что ваша реализация LU разложения работает с блочными матрицами.
4. Реализуйте LUP разложение с перестановкой строк. Предъявите матрицу, на которой LUP разложение работает, а LU - нет.
5. Реализуйте метод прогонки и реализуйте метод `Matrix.solve` для решения линейных систем уравнений.
6. Реализуйте класс `SymmetricMatrix`, хранящий симметричные матрицы. Убедитесь, что метод `Matrix.lu` корректно работает с этим классом. Модифицируйте этот метод для класса `SymmetricMatrix` так, чтобы он использовал симметричность матрицы и работал в два раза быстрее.
7. Как влияет симметричность матрицы на устойчивость LU разложения?
8. Реализуйте класс `BandMatrix` для хранения ленточных матриц. Убедитесь в работоспособности методов `lu` и `solve`.
9. Воспользуйтесь реализованными классами для решения уравнения Пуассона $\Delta f=g$, использую операцию Лапласа из предыдущей лабораторной.

In [1]:
import numpy as np

In [2]:
class TextBlock:
    def __init__(self, rows):
        assert isinstance(rows, list)
        self.rows = rows
        self.height = len(self.rows)
        self.width = max(map(len,self.rows))
        
    @classmethod
    def from_str(_cls, data):
        assert isinstance(data, str)
        return TextBlock( data.split('\n') )
        
    def format(self, width=None, height=None):
        if width is None: width = self.width
        if height is None: height = self.height
        return [f"{row:{width}}" for row in self.rows]+[' '*width]*(height-self.height)
    
    @staticmethod
    def merge(blocks):
        return [" ".join(row) for row in zip(*blocks)]
    
class Matrix:
    """Общий предок для всех матриц."""
    @property
    def shape(self):
        raise NotImplementedError
    
    @property
    def dtype(self):
        raise NotImplementedError
    
    @property 
    def width(self):
        return self.shape[1]
    
    @property 
    def height(self):
        return self.shape[0]    
        
    def __repr__(self):
        """Возвращает текстовое представление для матрицы."""
        text = [[TextBlock.from_str(f"{self[r,c]}") for c in range(self.width)] for r in range(self.height)]
        width_el = np.array(list(map(lambda row: list(map(lambda el: el.width, row)), text)))
        height_el = np.array(list(map(lambda row: list(map(lambda el: el.height, row)), text)))
        width_column = np.max(width_el, axis=0)
        width_total = np.sum(width_column)
        height_row = np.max(height_el, axis=1)
        result = []
        for r in range(self.height):
            lines = TextBlock.merge(text[r][c].format(width=width_column[c], height=height_row[r]) for c in range(self.width))
            for l in lines:
                result.append(f"| {l} |")
            if len(lines)>0 and len(lines[0])>0 and lines[0][0]=='|' and r<self.height-1:
                result.append(f'| {" "*(width_total+self.width)}|')
        return "\n".join(result)
    
    def empty_like(self, width=None, height=None):
        raise NotImplementedError
    
    def __getitem__(self, key):
        raise NotImplementedError
    
    def __setitem__(self, key, value):
        raise NotImplementedError
        
    def __add__(self, other):
        if isinstance(other, Matrix):
            assert self.width==other.width and self.height==other.height, f"Shapes does not match: {self.shape} != {other.shape}"
            matrix = self.empty_like()
            for r in range(self.height):
                for c in range(self.width):
                    matrix[r,c] = self[r,c] + other[r,c]
            return matrix
        return NotImplemented
    
    def __sub__(self, other):
        if isinstance(other, Matrix):
            assert self.width==other.width and self.height==other.height, f"Shapes does not match: {self.shape} != {other.shape}"
            matrix = self.empty_like()
            for r in range(self.height):
                for c in range(self.width):
                    matrix[r,c] = self[r,c] - other[r,c]
            return matrix
        return NotImplemented

    def __mul__(self, other):
        return self.__matmul__(other)
    
    def __matmul__(self, other):
        if isinstance(other, Matrix):
            assert self.width==other.height, f"Shapes does not match: {self.shape} != {other.shape}"
            matrix = self.empty_like()
            for r in range(self.height):
                for c in range(other.width):
                    acc = None
                    for k in range(self.width):
                        add = self[r,k]*other[k,c]
                        acc = add if acc is None else acc+add
                    matrix[r,c] = acc
            return matrix
        return NotImplemented
    
    def inverse(self):
        raise NotImplementedError
        
    def invert_element(self, element):
        if isinstance(element, float):
            return 1/element
        if isinstance(element, Fraction):
            return 1/element
        if isinstance(element, Matrix):
            return element.inverse()
        raise TypeError
        

class FullMatrix(Matrix):
    """
    Заполненная матрица с элементами произвольного типа.
    """
    def __init__(self, data):
        """
        Создает объект, хранящий матрицу в виде np.ndarray `data`.
        """
        assert isinstance(data, np.ndarray)
        self.data = data

    def empty_like(self, width=None, height=None):
        dtype = self.data.dtype
        if width is None:
            width = self.data.shape[1]
        if height is None:
            height = self.data.shape[0]       
        data = np.empty((height,width), dtype=dtype)
        return FullMatrix(data)
        
    @classmethod
    def zero(_cls, height, width, default=0):
        """
        Создает матрицу размера `width` x `height` со значениями по умолчанию `default`.
        """
        data = np.empty((height, width), dtype=type(default))
        data[:] = default
        return FullMatrix(data)
                    
    @property
    def shape(self):
        return self.data.shape
    
    @property
    def dtype(self):
        return self.data.dtype
        
    def __getitem__(self, key):
        row, column = key
        return self.data[row, column]
    
    def __setitem__(self, key, value):
        row, column = key
        self.data[row, column] = value
        

In [3]:
m = FullMatrix.zero(3,5,0)
print(m)
print(m.shape)
print(m.dtype)

| 0 0 0 0 0 |
| 0 0 0 0 0 |
| 0 0 0 0 0 |
(3, 5)
int64


In [4]:
from fractions import Fraction
m = FullMatrix.zero(5,7,Fraction(0,1))
for i in range(m.height):
    for j in range(m.width):
        m[i,j] = Fraction(i+1,j+1)
d = FullMatrix.zero(7,6,Fraction(0,1))
for i in range(min(d.height,d.width)):
        d[i,i] = Fraction(i,1)        
print(m)
print(m.shape)
print(m.dtype)
print(f"m+m", m+m)
print(f"m*d", m*d)

| 1 1/2 1/3 1/4 1/5 1/6 1/7 |
| 2 1   2/3 1/2 2/5 1/3 2/7 |
| 3 3/2 1   3/4 3/5 1/2 3/7 |
| 4 2   4/3 1   4/5 2/3 4/7 |
| 5 5/2 5/3 5/4 1   5/6 5/7 |
(5, 7)
object
m+m | 2  1 2/3  1/2 2/5 1/3 2/7  |
| 4  2 4/3  1   4/5 2/3 4/7  |
| 6  3 2    3/2 6/5 1   6/7  |
| 8  4 8/3  2   8/5 4/3 8/7  |
| 10 5 10/3 5/2 2   5/3 10/7 |
m*d | 0 1/2 2/3  3/4  4/5  5/6  None |
| 0 1   4/3  3/2  8/5  5/3  None |
| 0 3/2 2    9/4  12/5 5/2  None |
| 0 2   8/3  3    16/5 10/3 None |
| 0 5/2 10/3 15/4 4    25/6 None |


In [5]:
b = FullMatrix.zero(3,3,m)
print(b)

| | 1 1/2 1/3 1/4 1/5 1/6 1/7 | | 1 1/2 1/3 1/4 1/5 1/6 1/7 | | 1 1/2 1/3 1/4 1/5 1/6 1/7 | |
| | 2 1   2/3 1/2 2/5 1/3 2/7 | | 2 1   2/3 1/2 2/5 1/3 2/7 | | 2 1   2/3 1/2 2/5 1/3 2/7 | |
| | 3 3/2 1   3/4 3/5 1/2 3/7 | | 3 3/2 1   3/4 3/5 1/2 3/7 | | 3 3/2 1   3/4 3/5 1/2 3/7 | |
| | 4 2   4/3 1   4/5 2/3 4/7 | | 4 2   4/3 1   4/5 2/3 4/7 | | 4 2   4/3 1   4/5 2/3 4/7 | |
| | 5 5/2 5/3 5/4 1   5/6 5/7 | | 5 5/2 5/3 5/4 1   5/6 5/7 | | 5 5/2 5/3 5/4 1   5/6 5/7 | |
|                                                                                           |
| | 1 1/2 1/3 1/4 1/5 1/6 1/7 | | 1 1/2 1/3 1/4 1/5 1/6 1/7 | | 1 1/2 1/3 1/4 1/5 1/6 1/7 | |
| | 2 1   2/3 1/2 2/5 1/3 2/7 | | 2 1   2/3 1/2 2/5 1/3 2/7 | | 2 1   2/3 1/2 2/5 1/3 2/7 | |
| | 3 3/2 1   3/4 3/5 1/2 3/7 | | 3 3/2 1   3/4 3/5 1/2 3/7 | | 3 3/2 1   3/4 3/5 1/2 3/7 | |
| | 4 2   4/3 1   4/5 2/3 4/7 | | 4 2   4/3 1   4/5 2/3 4/7 | | 4 2   4/3 1   4/5 2/3 4/7 | |
| | 5 5/2 5/3 5/4 1   5/6 5/7 | | 5 5/2 5/3 5/4 1   5/6 5/7 